<a href="https://colab.research.google.com/github/cyamize/bayesian/blob/main/bayesian%EC%9D%B4%EB%A1%A0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import numpy as np
import pandas as pd
from nltk.corpus import stopwords
import string
import nltk

In [5]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [6]:
df = pd.read_csv('./spam.csv')
df

,Unnamed: 0,num,label,text,label_num
0,0,1,spam,your free lottery,1
1,1,2,spam,free lottery free you,1
2,2,3,ham,your free apple,0
3,3,4,ham,free to contact me,0
4,4,5,ham,I won award,0
5,5,6,spam,my lottery ticket,1


In [7]:
# 데이터에서 유의마한 단어 토큰만 주어진 text에서 선별한다
# i, my, me, over, on 등 조사나 접미사 같은 불용어(stopwords)는 제거한 후 사용해야하나, 
# 예제의 데이터셋은 데이터양이 많지 않으므로 생략한다

In [8]:
def process_text(text):

  # text에서 구두점을 삭제한다
  nopunc = [char for char in text if char not in string.punctuation]
  nopunc = ''.join(nopunc)

  # text에서 무의미한 단어(접미사, 조사 등)는 삭제합니다
  cleaned_words = [word for word in nopunc.split() 
                   if word.lower() not in stopwords.words('english')]
  return cleaned_words



In [9]:
df['text'].head()

0         your free lottery
1    free lottery free you 
2          your free apple 
3       free to contact me 
4               I won award
Name: text, dtype: object

In [10]:
# process_text함수를 적용하여 데이터 세트의 텍스트 데이터를 토큰화한다.
df['text'].apply(process_text)



0          [free, lottery]
1    [free, lottery, free]
2            [free, apple]
3          [free, contact]
4                  [award]
5        [lottery, ticket]
Name: text, dtype: object

In [11]:
# text를 토큰 수의 행렬로 변환
from sklearn.feature_extraction.text import CountVectorizer

message_bow = CountVectorizer(analyzer = process_text).fit_transform
(df['text'])



0         your free lottery
1    free lottery free you 
2          your free apple 
3       free to contact me 
4               I won award
5        my lottery ticket 
Name: text, dtype: object

In [16]:
# 행렬변환

from sklearn.feature_extraction.text import CountVectorizer
msg_bow = CountVectorizer(analyzer = process_text).fit_transform(df['text'])

In [17]:
msg_bow

<6x6 sparse matrix of type '<class 'numpy.int64'>'
	with 11 stored elements in Compressed Sparse Row format>

In [18]:
# 데이터를 80%의 training과 20%의 testing 데이터셋으로 분리한다.
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(msg_bow, df['label_num'], test_size = 0.20, random_state = 0)



In [19]:
# 다항식  나이브베이즈 모델을 만들고 훈련
from sklearn.naive_bayes import MultinomialNB
classifier = MultinomialNB()
classifier.fit(X_train, y_train)


MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [20]:

# 데이터셋 분류에 대한 예측 및 실제 관측 값을 보여준다
print(classifier.predict(X_train)) # 예측 값 출력
print(y_train.values)   # 실제 관측 괎 출력

[1 0 1 0]
[1 0 1 0]


In [22]:
# 학습 데이터셋에서 모델의 정확도를 표현
# 모델 정확도
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix, accuracy_score

In [23]:
pred = classifier.predict(X_train) # 모델에 관한 예측 값 출력

In [24]:
# 사이킷런 패키지의 metrics 패키지에서 정밀도, 재현율, F1 점수를 구함
print(classification_report(y_train,pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00         2
           1       1.00      1.00      1.00         2

    accuracy                           1.00         4
   macro avg       1.00      1.00      1.00         4
weighted avg       1.00      1.00      1.00         4



In [25]:
# 혼동행렬로 표시

confusion_matrix(y_train, pred)

array([[2, 0],
       [0, 2]])

In [26]:
# 실제 관측값 출력
accuracy_score(y_train, pred)

1.0

In [27]:
y_test.values

array([1, 0])

In [28]:
# 테스트 데이터셋에서 모델을 평가
pred = classifier.predict(X_test)

In [29]:
print(classification_report(y_test, pred))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         1
           1       0.50      1.00      0.67         1

    accuracy                           0.50         2
   macro avg       0.25      0.50      0.33         2
weighted avg       0.25      0.50      0.33         2



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [30]:
confusion_matrix(y_test, pred)

array([[0, 1],
       [0, 1]])

In [31]:
accuracy_score(y_test, pred)

0.5